# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
1/01/2022	A Capsule Network for�Hierarchical Multi-label Image Classification	"Structural, Syntactic, and Statistical Pattern Recognition"	This paper is about the multi-label image classification using a capsule network.	"<i>K. T. Noor, A. Robles-Kelly, and B. Kusy, �A Capsule Network for�Hierarchical Multi-label Image Classification�, in Structural, Syntactic, and Statistical Pattern Recognition, A. Krzyzak, C. Y. Suen, A. Torsello, and N. Nobile, Eds., in Lecture Notes in Computer Science. Cham: Springer International Publishing, 2022, pp. 163�172. doi: 10.1007/978-3-031-23028-8_17.</i>"	ML-CapsNet	http://ktnoor.github.io/files/ml-capsnet.pdf	
1/10/2024	H-CapsNet: A capsule network for hierarchical image classification	Journal 1	This paper is about the number 2. The number 3 is left for future work.	"Your Name, You. (2010). ""Paper Title Number 2."" <i>Journal 1</i>. 1(2)."	paper-title-number-2	http://academicpages.g

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [5]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,1/01/2022,A Capsule Network for Hierarchical Multi-label...,"Structural, Syntactic, and Statistical Pattern...",This paper is about the multi-label image clas...,"<i>K. T. Noor, A. Robles-Kelly, and B. Kusy, A...",ML-CapsNet,http://ktnoor.github.io/files/ml-capsnet.pdf,NaN
1,1/03/2024,H-CapsNet: A capsule network for hierarchical ...,Pattern Recognition,This paper is about hierarchical multi-label i...,"<i>K. T. Noor and A. Robles-Kelly, H-CapsNet: ...",H-CapsNet,http://ktnoor.github.io/files/h-capsnet.pdf,NaN
2,1/11/2023,A Bottom-Up Capsule Network for Hierarchical I...,2023 International Conference on Digital Image...,This paper is about the hierarchical image cla...,"<i>K. T. Noor, A. Robles-Kelly, L. Y. Zhang, a...",BUH-CapsNet,http://ktnoor.github.io/files/buh-capsnet.pdf,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [6]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [7]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [8]:
!ls ../_publications/

2022-ML-CapsNet.md  2023-BUH-CapsNet.md  2024-H-CapsNet.md  2024-hd-capsnet.md
2022-ml-capsnet.md  2023-buh-capsnet.md  2024-h-capsnet.md


In [9]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
